Reconocedor de lenguaje de señas Argentino entrenado solo con el dataset argentino.

In [1]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

import numpy as np
import matplotlib.pyplot as plt

2025-08-04 16:22:41.453778: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-04 16:22:41.546665: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-04 16:22:41.615893: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754335361.691845   13483 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754335361.713158   13483 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754335361.856183   13483 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

Cargamos el modelo de manera que sea entrenable. No incluímos su última capa para poder establecer nuestras propias clases.

In [2]:
# Cargamos InceptionV3
base_model = InceptionV3(weights = 'imagenet',       # Pre-entrenado con ImageNet
                         include_top = False,        # Sin incluir su capa de clasificacion con 1000 clases para poder hacer fine-tuning 
                         input_shape = (299, 299, 3) # Necesario cuando no incluimos la ultima capa
                        )

# Inicialmente congelamos todas las capas, despues descongelaremos las que nos interesen
base_model.trainable = False

2025-08-04 16:22:45.603006: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Importamos los datasets y hacemos un split.

In [3]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Extraemos los nombres de todas las imagenes que vamos a utilizar
image_dir = 'lsa16_segmented/'
filenames = [f for f in os.listdir(image_dir)]

# Y de cada una extraemos su clase, que viene dada por el primer numero del nombre
labels = [int(f.split('_')[0]) - 1 for f in filenames]   # Le restamos 1 a los labels para que esten en rango [0, 16) en vez de [1, 16]

# Y creamos un dataframe que asocia a cada nombre de archivo su clase.
df = pd.DataFrame({'filename': filenames, 'class': labels})

Preprocesamos las imagenes para adecuarlas al formato de ImageNet

In [4]:
# Separamos al dataset en train y validacion.
train_df, val_df = train_test_split(df,
                                    test_size=0.2,
                                    stratify=df['class'],   # Hace que se mantengan las proporciones de las clases luego del split
                                    random_state=42)

Creamos un *pipeline* de datos de *TensorFlow*,

In [5]:
# Definimos una funcion que dado un filename devuelve su imagen y su clase o label
def load_and_preprocess(image_path, label):

    # Leemos el archivo y lo decodificamos en RGB
    img = tf.io.read_file(image_dir + image_path)
    img = tf.image.decode_jpeg(img, channels=3) 
    
    # Lo preprocesamos para InceptionV3
    img = tf.image.resize(img, [299, 299])
    img = preprocess_input(img)  # Obs. que preprocess_input es una funcion de inception_v3 en particular
    
    return img, label

# Usamos un batch_size de TensorFlow estandar
batch_size = 32

# 1. Cargamos el dataframe
ds = tf.data.Dataset.from_tensor_slices((train_df['filename'].values, train_df['class'].values))

# 2. Le mappeamos el preprocesamiento a cada entrada, paralelizando
ds = ds.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)

# 3. Mezclamos para randomizar el orden de las muestras
ds = ds.shuffle(buffer_size=len(train_df))

# 4. Usamos el batch_size estandar
ds = ds.batch(batch_size)

# 5. Permitimos el prefetching del proximo batch
train_ds = ds.prefetch(tf.data.AUTOTUNE)                                                       

# Y repetimos lo mismo para el conjunto de validacion
val_ds = tf.data.Dataset.from_tensor_slices((val_df['filename'].values, val_df['class'].values))\
           .map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE) \
           .batch(batch_size) \
           .prefetch(tf.data.AUTOTUNE)

In [6]:
print(val_df['class'].values)

[10  0  9  0  3  6  5  0  3  5  3 12  9  2  8  1 12  6  2  2  5  2 10 11
 11 13  3  2  0  7  8 12  8 11 14  7  4 10  4 12  6 14 10 14 13  2 11  7
  9 14 10 15  8 15  8 12 13 13  3  6  3  8  2  1  9  4  3  9  8  0 10  4
  8  7 12  0 13  4  5  7  5  9  1  7 10 14  5  7 11  7 11  1 10 13  6  5
 13  6  4  6 11  5  1 11  8 12 14  7  7  5 14  4  2 15 13 13  6 11  2 14
 15 15  0  4  9  6  0  8 10  0 12 14  6  2  4 15 12  9  1  1  3  5 12 15
  1 14  9  0 11  1  3 15  3 13  9  4 10 15 15  1]


In [7]:
# Definimos la ultima capa para que prediga acorde a nuestras clases
num_classes = train_df['class'].nunique()
print(f"Número de clases: {num_classes}")

x = base_model.output  # x es la salida del modelo hasta ahora
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Regularización para evitar overfitting

# Y creamos una nueva capa de salida que tome como input a la anterior y clasifique en num_classes clases
predictions = Dense(num_classes, activation='softmax')(x)  


model = Model(inputs=base_model.input, outputs=predictions)

Número de clases: 16


Configuramos el modelo usando momento adaptativo y *sparse categorial cross-entropy* pues es adecuada para clasificacion multiclase con enteros.

In [8]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',  # Para etiquetas enteras
    metrics=['accuracy']
)

Entrenemos el modelo con los nuevos datos y guardamos sus pesos

In [9]:
history = model.fit(train_ds, epochs = 1, validation_data = val_ds, verbose = 2) #, batch_size = 20) # borrar batch_size? 
model.save_weights("arg_only_interpreter_final.weights.h5")

2025-08-04 16:22:52.670814: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 177020928 exceeds 10% of free system memory.
2025-08-04 16:22:52.861742: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 123887616 exceeds 10% of free system memory.
2025-08-04 16:22:54.268510: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 177020928 exceeds 10% of free system memory.
2025-08-04 16:22:54.476856: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 123887616 exceeds 10% of free system memory.
2025-08-04 16:22:55.837792: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 177020928 exceeds 10% of free system memory.


20/20 - 46s - 2s/step - accuracy: 0.2406 - loss: 2.5064 - val_accuracy: 0.5375 - val_loss: 1.4941


In [10]:
img_path = 'asl_dataset/a/hand1_a_bot_seg_5_cropped.jpeg'  
img = image.load_img(img_path, target_size=(299, 299)) # La carga en img y le hace resize a 299x299
img_array = image.img_to_array(img)                    # La convierte a array de NumPy con dimensiones (299, 299, 3)
img_array = np.expand_dims(img_array, axis=0)          # Agrega una dimension mas al array, haciendolo (1, 299, 299, 3) para batching
img_array = preprocess_input(img_array)                # Matchea la representacion de la imagen a como la espera ImageNet (ej. mappea 0-255 a -1,1, cambia de RGB a BGR)

# Predict
predictions = model.predict(img_array)
decoded_predictions = decode_predictions(predictions, top=5)[0]

# Display results
plt.imshow(img)
plt.axis('off')
plt.show()

print("Top 5 Predictions:")
for i, (_, label, prob) in enumerate(decoded_predictions):
    print(f"{i + 1}: {label} ({prob * 100:.2f}%)")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


ValueError: `decode_predictions` expects a batch of predictions (i.e. a 2D array of shape (samples, 1000)). Received array with shape: (1, 16)